In [70]:
from dotenv import load_dotenv
import os
import google.generativeai as genai

# Load .env file
load_dotenv()

# Get API key
gemini_key = os.getenv("GEMINI_API_KEY")

In [72]:

genai.configure(api_key=gemini_key)

model_gemini = genai.GenerativeModel("gemini-2.5-flash")

prompt = """
The user has symptoms: fever, cough, fatigue.
Suggest possible diseases and basic advice.
"""

response = model_gemini.generate_content(prompt)
print(response.text)


These symptoms (fever, cough, fatigue) are very common and can be indicative of several conditions. It's important to remember that I am an AI and cannot diagnose. This information is for general guidance only, and you should **consult a healthcare professional for an accurate diagnosis and treatment plan.**

Here are some possible diseases and basic advice:

### Possible Diseases

1.  **Common Cold:** A mild viral infection of the nose and throat.
    *   **Symptoms:** Usually milder, with a low-grade fever, runny/stuffy nose, sore throat, cough, and fatigue.
    *   **Onset:** Gradual.

2.  **Influenza (Flu):** A more severe viral infection of the respiratory system.
    *   **Symptoms:** Often more abrupt and intense than a cold, with higher fever, body aches, headache, severe fatigue, dry cough, and sore throat.
    *   **Onset:** Sudden.

3.  **COVID-19 (Coronavirus Disease 2019):** A respiratory illness caused by the SARS-CoV-2 virus.
    *   **Symptoms:** Highly variable, but co

In [59]:
predicted_disease = "Influenza"

prompt = f"""
The model predicted the disease: {predicted_disease}.
Suggest a short treatment plan, medicines, and a doctor specialization to visit.
"""

response = model_gemini.generate_content(prompt)
print(response.text)

Here's a short treatment plan for Influenza:

**Treatment Plan:**

1.  **Rest:** Get plenty of bed rest to allow your body to recover.
2.  **Hydration:** Drink clear fluids (water, broth, herbal tea, juice) frequently to prevent dehydration, especially if you have a fever.
3.  **Symptomatic Relief:** Address symptoms like fever, aches, and cough.
4.  **Monitor Symptoms:** Watch for worsening symptoms (e.g., difficulty breathing, chest pain, confusion) and seek immediate medical attention if they occur.
5.  **Prevent Spread:** Practice good hand hygiene, cover coughs/sneezes, and avoid close contact with others.

**Medicines:**

*   **Over-the-Counter (OTC):**
    *   **Pain/Fever:** Acetaminophen (e.g., Tylenol) or Ibuprofen (e.g., Advil, Motrin).
    *   **Cough/Cold Symptoms:** Cough suppressants (e.g., Dextromethorphan), expectorants (e.g., Guaifenesin), or decongestants (use with caution, especially if you have high blood pressure).
    *   **Sore Throat:** Lozenges or gargles.
*  

In [39]:
TRAIN_FEATURES = [
    "Fever", "Cough", "Fatigue", "Difficulty Breathing",
    "Gender", "Blood Pressure", "Cholesterol Level", "Outcome Variable", "Age"
]

BINARY_SYMPTOMS = ["Fever", "Cough", "Fatigue", "Difficulty Breathing"]
MULTI_SYMPTOMS = ["Gender", "Blood Pressure", "Cholesterol Level", "Outcome Variable"]

prompt_template = """
Extract symptoms and their values from the user text. Return a JSON object exactly containing keys:
{keys}
Rules:
- Use 'Yes', 'No', or 'Unknown' for binary features.
- Age as integer if present.
- Output only valid JSON.
Input: {user_text}
"""

PROMPT = PromptTemplate(
    input_variables=["keys", "user_text"],
    template=prompt_template
)


In [66]:
import pickle

with open("finalized_model.pkl", "rb") as file:
    loaded = pickle.load(file)

# Extract the model from the tuple
model = loaded[0]


In [67]:
import pickle

with open("label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

prediction = model.predict(user_input)
predicted_disease = label_encoder.inverse_transform(prediction.astype(int))

print("Predicted disease:", predicted_disease[0])


Predicted disease: 1


In [54]:
import pickle

with open("finalized_model.pkl", "rb") as file:
    loaded = pickle.load(file)

print(type(loaded))


<class 'sklearn.ensemble._forest.RandomForestClassifier'>


In [55]:
print(loaded)


RandomForestClassifier(random_state=0)


In [56]:
def predict_disease(symptoms_input):
    # Predict encoded label
    prediction = model.predict(symptoms_input)
    
    # Decode the label to actual disease name
    predicted_disease = label_encoder.inverse_transform(prediction.astype(int))
    
    return predicted_disease[0]


In [64]:
user_input = [[1, 1, 0, 0, 1, 0, 0, 1, 0, 0]]  # Sample test case

predicted_disease = predict_disease(user_input)
print("🧠 Predicted Disease:", predicted_disease)


🧠 Predicted Disease: 1


In [58]:
proba = model.predict_proba(user_input)
confidence = round(max(proba[0]) * 100, 2)

print(f"🧠 Predicted Disease: {predicted_disease} ({confidence}% confidence)")


🧠 Predicted Disease: 1 (100.0% confidence)


In [60]:
import pickle

with open("label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

print(label_encoder.classes_[:10])  # show first 10 disease names
print("Total encoded classes:", len(label_encoder.classes_))


[0 1 2 3 4 5 6 7 8 9]
Total encoded classes: 116


In [61]:
pred = model.predict(user_input)
print("Raw prediction:", pred)
print("Type:", type(pred[0]))


Raw prediction: [1.]
Type: <class 'numpy.float64'>


In [62]:
import numpy as np

# predict
pred = model.predict(user_input)

# ensure integer type
pred_int = np.array(pred, dtype=int)

# decode to disease name
predicted_disease = label_encoder.inverse_transform(pred_int)

print("🧠 Predicted Disease:", predicted_disease[0])


🧠 Predicted Disease: 1
